In [23]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from collections import Counter

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
def label_map(label):
    if label == "neutral":
        return 0
    elif label == "anger":
        return 1
    elif label == "joy":
        return 2
    elif label == "surprise":
        return 3
    elif label == "sadness":
        return 4
    elif label == "disgust":
        return 5
    elif label == "fear":
        return 6
    
def encode(text, word2index, label, N):
    tokenized = word_tokenize(text)
    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return (encoded,label)

def encode_test(text, word2index, N):
    tokenized = word_tokenize(text)
    for i,word in enumerate(tokenized):
        if word2index.get(word)==None:
            tokenized[i]='unk'

    encoded = [0]*N
    enc1 = [word2index.get(word) for word in tokenized]

    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    for batch in iterator:
        text = batch[0].to(device)
        target = batch[1]
        target = target.type(torch.LongTensor)
        target = target.to(device)
        preds = model(text)
        loss = criterion(preds, target)
        _, pred = torch.max(preds, 1)
        acc = accuracy_score(pred.tolist(), target.tolist())

        epoch_loss += loss.item()
        epoch_acc += acc

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
train_df = pd.read_csv('train_HW2dataset.csv')
dev_df = pd.read_csv('dev_HW2dataset.csv')

train_df = train_df[['Emotion','Utterance']]
dev_df = dev_df[['Emotion','Utterance']]

train_set = list(train_df.to_records(index=False))
dev_set = list(dev_df.to_records(index=False))

counts = Counter()
for ds in [train_set, dev_set]:
    for label,text in ds:
        counts.update(word_tokenize(text))

word2index = {'unk':0}
for i,word in enumerate(counts.keys()):
    word2index[word] = i+1
index2word = {v:k for k,v in word2index.items()}

train_encoded = [(encode(Utterance,word2index,label_map(label),12)) for label, Utterance in train_set]
dev_encoded   = [(encode(Utterance,word2index,label_map(label),12)) for label, Utterance in dev_set]

train_x = np.array([tweet for tweet, label in train_encoded])
train_y = np.array([label for tweet, label in train_encoded])
dev_x = np.array([tweet for tweet, label in dev_encoded])
dev_y = np.array([label for tweet, label in dev_encoded])

batch_size = 32

train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
dev_ds = TensorDataset(torch.from_numpy(dev_x), torch.from_numpy(dev_y))

train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
dev_dl = DataLoader(dev_ds, shuffle=True, batch_size=batch_size, drop_last=True)

In [35]:
src_vocab_size = len(word2index)      
dimension_model = 32                             
num_layers = 5                       
hidden_size = 30                         
linear_hidden_size = 10               
classes = 7                          
dropout = 0.2                            
lr = 1e-3      

In [36]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.embed = torch.nn.Embedding(src_vocab_size, dimension_model)                        
        self.lstm = torch.nn.LSTM(input_size=dimension_model, hidden_size=hidden_size,num_layers=num_layers,dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, linear_hidden_size)
        self.linear1 = torch.nn.Linear(linear_hidden_size, classes)
    def forward(self,data):
        x = self.embed(data)                          
        x,(h_n, c_n) = self.lstm(x.transpose(0, 1))                                       
        x = self.linear(x[-1])                          
        x = self.linear1(x)                            
        return x  

In [37]:
model = LSTM().to(device)                           
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 

best_acc = 0
for epoch in range(10):
    train_loss, train_acc = train(model, train_dl,
                                  optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, dev_dl,
                                     criterion)

    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc * 100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc * 100:.2f}%')    
    if best_acc <= valid_acc:
        best_acc = valid_acc
        PATH=f"epoch{epoch+1}_val.accuracy{valid_loss:.3f}%.pt"
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': valid_loss,
                }, PATH)

Epoch: 01, Train Loss: 1.513, Train Acc: 46.55%, Val. Loss: 1.521, Val. Acc: 44.93%
Epoch: 02, Train Loss: 1.389, Train Acc: 52.46%, Val. Loss: 1.495, Val. Acc: 46.60%
Epoch: 03, Train Loss: 1.351, Train Acc: 53.20%, Val. Loss: 1.497, Val. Acc: 46.25%
Epoch: 04, Train Loss: 1.306, Train Acc: 54.48%, Val. Loss: 1.485, Val. Acc: 47.01%
Epoch: 05, Train Loss: 1.253, Train Acc: 56.64%, Val. Loss: 1.518, Val. Acc: 47.64%
Epoch: 06, Train Loss: 1.203, Train Acc: 58.71%, Val. Loss: 1.572, Val. Acc: 47.43%
Epoch: 07, Train Loss: 1.162, Train Acc: 59.91%, Val. Loss: 1.631, Val. Acc: 46.94%
Epoch: 08, Train Loss: 1.124, Train Acc: 61.17%, Val. Loss: 1.624, Val. Acc: 46.74%
Epoch: 09, Train Loss: 1.092, Train Acc: 62.56%, Val. Loss: 1.603, Val. Acc: 46.67%
Epoch: 10, Train Loss: 1.057, Train Acc: 64.16%, Val. Loss: 1.645, Val. Acc: 47.57%


In [32]:
test_df = pd.read_csv('test_HW2dataset.csv')
test_df = test_df[['Utterance']]
test_set = test_df.values.tolist()
test_encoded=[]
for sentence in test_set:
    test_encoded += [encode_test(Utterance, word2index, 10) for Utterance in sentence]
test_x = np.array(test_encoded)
test_ds = TensorDataset(torch.from_numpy(test_x))
test_dl = DataLoader(test_ds, shuffle=False)
 
model = LSTM().to(device)                               
criterion = torch.nn.CrossEntropyLoss()                  
optimizer = torch.optim.Adam(model.parameters(),lr=lr) 
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
predict=[]
for deta in test_dl:
    text = deta[0].to(device)
    preds = model(text)
    _, pred = torch.max(preds, 1)
    predict.append(pred.item())

In [33]:
ans = pd.DataFrame(columns=["index","emotion"])
for ind, p in enumerate(predict):
    ans.loc[ind] = [ind,p]
ans.to_csv("predict.csv",index=False)